In [1]:
import paddle
import paddle.fluid as fluid
import paddle.dataset.imdb as imdb
paddle.enable_static()
def rnn_net(ipt, input_dim):
    emb=fluid.embedding(input=ipt,size=[input_dim,128],is_sparse=True)
    sentence=fluid.layers.fc(input=emb, size=128, act='tanh')
    rnn=fluid.layers.DynamicRNN()
    with rnn.block():
        word=rnn.step_input(sentence)
        prev=rnn.memory(shape=[128])
        hidden=fluid.layers.fc(input=[word,prev],size=128,act='relu')
        rnn.update_memory(prev,hidden)
        rnn.output(hidden)
    last=fluid.layers.sequence_last_step(rnn())
    out=fluid.layers.fc(input=last, size=2,act='softmax')
    return out
words=fluid.data(name='words',shape=[None,1], dtype='int64', lod_level=1)#这个是输入层
word_dict=imdb.word_dict()
dict_dim=len(word_dict)
model=rnn_net(words, dict_dim)#相当于输出层
infer_program=fluid.default_main_program().clone(for_test=True)

label=fluid.data(name='label',shape=[None,1],dtype='int64')
cost=fluid.layers.cross_entropy(input=model,label=label)
avg_cost=fluid.layers.mean(cost)
acc=fluid.layers.accuracy(input=model,label=label)
test_program=fluid.default_main_program().clone(for_test=True)
optimizer=fluid.optimizer.AdagradOptimizer(learning_rate=0.002)
opt=optimizer.minimize(avg_cost)

place=paddle.CUDAPlace(0)
exe=fluid.Executor(place)
exe.run(fluid.default_startup_program())

train_reader=paddle.batch(imdb.train(word_dict),batch_size=128)
test_reader=paddle.batch(imdb.test(word_dict),batch_size=128)
feeder=fluid.DataFeeder(place=place, feed_list=[words, label])

for pass_id in range(1):
    train_cost=0
    for batch_id, data in enumerate(train_reader()):
        train_cost,train_acc=exe.run(program=fluid.default_main_program(),
                                     feed=feeder.feed(data),
                                     fetch_list=[avg_cost, acc])
        if batch_id%10==0:
            print('Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f' %
                  (pass_id, batch_id, train_cost[0], train_acc[0]))
    test_costs=[]
    test_accs=[]
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc=exe.run(program=test_program,
                                   feed=feeder.feed(data),
                                   fetch_list=[avg_cost, acc])
        test_costs.append(test_cost[0])
        test_accs.append(test_acc[0])
    test_cost=(sum(test_costs)/len(test_costs))
    test_acc=(sum(test_accs)/len(test_accs))
    print('Test:%d, Cost:%0.5f, Accuracy:%0.5f' %
          (pass_id, test_cost, test_acc))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:687: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated

Pass:0, Batch:0, Cost:0.70243, Accuracy:0.22656
Pass:0, Batch:10, Cost:0.00001, Accuracy:1.00000
Pass:0, Batch:20, Cost:0.00005, Accuracy:1.00000
Pass:0, Batch:30, Cost:0.00003, Accuracy:1.00000
Pass:0, Batch:40, Cost:0.00004, Accuracy:1.00000
Pass:0, Batch:50, Cost:0.00001, Accuracy:1.00000
Pass:0, Batch:60, Cost:0.00001, Accuracy:1.00000
Pass:0, Batch:70, Cost:0.00000, Accuracy:1.00000
Pass:0, Batch:80, Cost:0.00004, Accuracy:1.00000
Pass:0, Batch:90, Cost:0.00004, Accuracy:1.00000
Pass:0, Batch:100, Cost:1.27651, Accuracy:0.00000
Pass:0, Batch:110, Cost:1.22711, Accuracy:0.00000
Pass:0, Batch:120, Cost:1.17444, Accuracy:0.00000
Pass:0, Batch:130, Cost:1.14945, Accuracy:0.00000
Pass:0, Batch:140, Cost:1.12816, Accuracy:0.00000
Pass:0, Batch:150, Cost:1.07141, Accuracy:0.00000
Pass:0, Batch:160, Cost:1.02571, Accuracy:0.00000
Pass:0, Batch:170, Cost:0.98311, Accuracy:0.00000
Pass:0, Batch:180, Cost:0.94634, Accuracy:0.00000
Pass:0, Batch:190, Cost:0.91952, Accuracy:0.00781
Test:0, Cos

大佬们train出来的都是80，我train出来50

In [16]:
import os
save_path='.//work//model//'
fluid.io.save_persistables(executor=exe, dirname=save_path)#用于初始化或者迁移学习
fluid.io.save_params(executor=exe, dirname=save_path)#只用于初始化，所以不常用
fluid.io.save_inference_model(dirname=save_path,
                            feeded_var_names=[words.name],
                            target_vars=[model],
                            executor=exe)#推理模型

['save_infer_model/scale_0.tmp_2']

In [6]:
import numpy as np
reviews_str=['awesome movie',
            'this is a terrible movie',
            'this is very bad']
reviews=[c.split() for c in reviews_str]
print(reviews)
UNK=word_dict['<unk>']
lod=[]
for c in reviews:
    lod.append([np.int64(word_dict.get(words.encode('utf-8'),UNK)) for words in c])
print(lod)
base_shape=[[len(c) for c in lod]]
print(base_shape)
tensor_words=fluid.create_lod_tensor(lod,base_shape,place)

[['awesome', 'movie'], ['this', 'is', 'a', 'terrible', 'movie'], ['this', 'is', 'very', 'bad']]
[[1163, 16], [9, 5, 2, 376, 16], [9, 5, 51, 81]]
[[2, 5, 4]]


In [8]:
results=exe.run(program=infer_program,
                feed={words.name:tensor_words},
                fetch_list=[model])
print(results[0])

[[0.5371809  0.46281907]
 [0.5688881  0.43111184]
 [0.568448   0.431552  ]]


In [18]:
exe.run(fluid.default_startup_program())#重新对执行器进行初始化
[infer_program,input_layer,output_layer]=fluid.io.load_inference_model(dirname=save_path,executor=exe)
results_by_saved_inference_model=exe.run(program=infer_program,
                                        feed={input_layer[0]:tensor_words},
                                        fetch_list=output_layer)#因为输入层和输出层保存时都是列表保存的，所以返回时也是列表
print(results_by_saved_inference_model)

[array([[0.5371809 , 0.46281907],
       [0.5688881 , 0.43111184],
       [0.568448  , 0.431552  ]], dtype=float32)]


用保存的预测模型进行推理

直接使用内存里的参数进行预测，程序退出后无法使用

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 